# Case Study 5.2 - 03 Permutations

In the final section of this case study we look at permutation on the content of text in order to understand what drives model output.

To reduce the complexity of the task we use the keywords determined by KeyBERT to focus on permutation of specific words.

This Notebook combines the following scripts:
* [Prepare Data CaseStudy_5.2_03-01.py](CaseStudy_5.2_03-01.py)
* [Permute CaseStudy_5.2_03-02.py](CaseStudy_5.2_03-02.py)
* [Evaluate CaseStudy_5.2_03-04.py](CaseStudy_5.2_03-04.py)

**Note:** You will need to run the script to generate embeddings using cloud compute.
[CaseStudy_5.2_03-03_permuted_embeddings.py](CaseStudy_5.2_03-03_permuted_embeddings.py)
  

## 03 - 01 Prepare a dataset

We want a dataset of records that have at least one of the identified keywords.

A larger analysis would require a more extensive keyword list that covers the complete dataset.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("data/clustered_data.csv")
test = df[df["cluster"]==0]
test = test[test["RANDOM"]>=0.8]

In [ ]:
# Filter to records that have at least one keyword
keywords = pd.read_csv("data/keywords.csv")
keywords = keywords['keyword'].to_list()
exp_keywords = ["[^a-zA-Z]" + x + "[^a-zA-Z]" for x in keywords]
expr = "|".join(exp_keywords)
records = test[test['text'].str.contains(expr, case=False)]

In [ ]:
test_ai = records[records["generated"]==1]
test_human = records[records["generated"]==0]

ai_set = test_ai.sample(200, axis=0)
human_set = test_human.sample(200, axis=0)

test_set = pd.concat([ai_set, human_set], ignore_index=True)
test_set['record_index'] = test_set.index
test_set.to_csv("data/test_sample_for_text_permutation.csv", index=False)

## 03 - 02 Permute data

In [ ]:
import pandas as pd
import json
import re

In [ ]:
df = pd.read_csv("data/test_sample_for_text_permutation.csv")
new_rows = []

In [ ]:
with open("data/keywords.json") as f:
   lookup = json.load(f)

In [ ]:
for index, row in df.iterrows():
   r = row['record_index']
   generated = row['generated']
   source = row['source']
   text = row['text']
   record = {"record":r, "source":source, "text":text, "generated":generated, "word":"NONE", "sub":"NONE"}
   new_rows.append(record)
   for k in lookup.keys():
      sub = lookup[k]
      pattern = r"([^a-zA-Z])" + k + r"([^a-zA-Z])"
      if re.search(pattern, text, re.IGNORECASE):
         replacement = r"\1"+sub+r"\2"
         new_text = re.sub(pattern, replacement, text, re.IGNORECASE)
         new_record = {"record":r, "source":source, "text":new_text, "generated":generated, "word":k, "sub":sub}
         new_rows.append(new_record)

new_df = pd.DataFrame(new_rows)

new_df.to_csv("data/permuted_text_samples.csv", index=False)

## 03 - 03 Evaluate

In this section we use evaluate the embeddings of the permuted text data.

Before running this section you will need to generate the embeddings for text in the file generated above.
Do so using the script: [CaseStudy_5.2_03-03_permuted_embeddings.py](CaseStudy_5.2_03-03_permuted_embeddings.py)


In [ ]:
import pandas as pd
import numpy as np
import pickle

In [ ]:
df = pd.read_csv("data/permuted_text_samples.csv")
embeddings = "permutation_embeddings.npy"

In [ ]:
def expand_array_col(df, col_name):
    expanded = df[col_name].apply(pd.Series)
    expanded.columns = [f'{col_name}_{i+1}' for i in range(expanded.shape[1])]
    df_expanded = pd.concat([df.drop(col_name, axis=1), expanded], axis=1)
    return df_expanded, expanded.columns

embs = np.load(embeddings, allow_pickle=True,)
df["embedding"] = list(embs)
newdf, cols = expand_array_col(df, "embedding")

In [ ]:
refs = newdf[newdf['word']=="NONE"].copy()
subs = newdf[newdf['word']!="NONE"].copy()

diff_cols = ["c_"+x.split("_")[1] for x in cols]

In [ ]:
def get_diffs(row, ref_row):
    refs = ref_row[cols]
    vals = row[cols]
    diffs = refs - vals
    return diffs.to_dict()

results = pd.DataFrame()

In [ ]:
for index, ref_row in refs.iterrows():
    subset = subs[subs['record']==ref_row['record']].copy()
    new_cols = subset.apply(lambda r: get_diffs(r, ref_row), axis=1, result_type='expand')
    subset[diff_cols] = new_cols
    results = pd.concat([results, subset], ignore_index=True)

results.to_csv("data/permuted_text_samples_with_embeddings.csv", index=False)


In [ ]:
words = list(subs['word'].unique())

word_results = []

In [ ]:
for w in words:
    subset = results[results['word']==w]
    mean_absolute_values = subset[diff_cols].abs().mean()
    mean_values = subset[diff_cols].mean()
    abs_vals = mean_absolute_values.to_dict()
    mean_vals = mean_values.to_dict()
    abs_c = mean_absolute_values.idxmax()
    mean_c = mean_values.idxmax()
    diffabs = abs_vals[abs_c]
    diffval = mean_vals[abs_c]
    record = {"word":w, "component":abs_c, "d_absolute":diffabs, "d_mean":diffval}
    word_results.append(record)

In [32]:
result_df = pd.DataFrame(word_results)
print(result_df.round(3).to_markdown(index=False))

| word         | component   |   d_absolute |   d_mean |
|:-------------|:------------|-------------:|---------:|
| hey          | c_33        |        0.001 |   -0.001 |
| knows        | c_309       |        0.025 |    0.024 |
| yeah         | c_309       |        0.013 |    0.013 |
| awesome      | c_309       |        0.028 |    0.028 |
| activities   | c_658       |        0.012 |   -0.012 |
| choices      | c_309       |        0.011 |    0.011 |
| teachers     | c_309       |        0.017 |   -0.011 |
| summer       | c_342       |        0.01  |    0.01  |
| schoolwork   | c_309       |        0.012 |   -0.001 |
| watching     | c_187       |        0.008 |    0.008 |
| generic_name | c_1         |        0     |    0     |
| texting      | c_309       |        0.02  |    0.014 |
| student_name | c_1         |        0     |    0     |
| teacher_name | c_1         |        0     |    0     |


In [ ]:
filename = 'xt_BERT_model.pkl'
xt_model = pickle.load(open(filename, 'rb'))

In [ ]:
X_refs = refs.loc[:,cols]
X_subs = subs.loc[:,cols]

In [ ]:
scores = xt_model.predict_proba(X_refs)
prob_ai = scores[:,1]
refs["prob_ai"] = prob_ai

In [ ]:
scores = xt_model.predict_proba(X_subs)
prob_ai = scores[:,1]
subs["prob_ai"] = prob_ai

In [ ]:
model_results = pd.DataFrame()

for index, ref_row in refs.iterrows():
    subset = subs[subs['record']==ref_row['record']].copy()
    ref_prob = ref_row["prob_ai"]
    subset["prob_diff"] = subset["prob_ai"] - ref_prob
    subset["abs_prob_diff"] = np.abs(subset["prob_diff"])
    model_results = pd.concat([model_results, subset], ignore_index=True)

In [33]:
agg_func = {"abs_prob_diff":"mean", "prob_diff":"mean"}
grpd = model_results.groupby("word").agg(agg_func).reset_index()
print(grpd.round(3).to_markdown(index=False))

| word         |   abs_prob_diff |   prob_diff |
|:-------------|----------------:|------------:|
| activities   |           0.011 |       0.002 |
| awesome      |           0.02  |      -0.018 |
| choices      |           0.008 |      -0.004 |
| generic_name |           0     |       0     |
| hey          |           0.002 |      -0.001 |
| knows        |           0.012 |      -0.006 |
| schoolwork   |           0.009 |      -0.003 |
| student_name |           0     |       0     |
| summer       |           0.007 |       0     |
| teacher_name |           0     |       0     |
| teachers     |           0.015 |       0.003 |
| texting      |           0.012 |      -0.005 |
| watching     |           0.008 |      -0.005 |
| yeah         |           0.009 |      -0.005 |
